In [24]:
import pandas as pd
import numpy as np
from sklearn.neighbors import LocalOutlierFactor
import math

filename = "breast-cancer-wisconsin.data.txt"

# Initialize datafrmae
dataframe = pd.read_csv(filename, header=None)
dataframe = dataframe.drop(0, 1)
columns = ['ClTh', 'UnCeSi', 'UnCeSh', 'MaAd', 'SiEpCeSi', 'BaNu', 
           'BlCh', 'NoNu', 'Mitoses', 'Class']
dataframe.columns = columns

# Clean out missing values
dataframe = dataframe[dataframe.BaNu != '?']
dataframe = dataframe.reset_index(drop=True)
dataframe = dataframe.astype(int)

# Clean out outliers
LOF = LocalOutlierFactor(n_neighbors = 200)
Local = LOF.fit_predict(dataframe.drop(['Class'], axis=1))
dataframe['LOF_det'] = Local

# Drop outlier column 
dataframe = dataframe[dataframe.LOF_det != -1]
dataframe = dataframe.drop(['LOF_det'], axis=1)
dataframe = dataframe.reset_index(drop=True)

# Change class values from 4,2 to 1,0
dataframe['Class'] = dataframe['Class'].replace(4, 1)
dataframe['Class'] = dataframe['Class'].replace(2, 0)

# Split data 80-20
len_data = len(dataframe.index)
len_split = math.ceil(len_data * 0.8)
dataTest = dataframe.head(len_data - len_split)
dataTrain = dataframe.head(len_split)

# Get X and Y Data
X_train = dataTrain.drop('Class', 1)
Y_train = dataTrain.Class
X_test = dataTest.drop('Class', 1)
Y_test = dataTest.Class

# print(X_test)
# print(Y_test)

# print(dataframe)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/neighbors/lof.py:236: FutureWarning: default contamination parameter 0.1 will change in version 0.22 to "auto". This will change the predict method behavior.
  FutureWarning)


In [25]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

LogReg = LogisticRegression()
LogReg.fit(X_train, Y_train)
# scores = cross_val_score(LogReg, X_train.values, Y_train.values, 5)
# print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

LogReg.score(X_test.values, Y_test.values)

# x_sample = X_data.iloc[3].values
# x_sample = x_sample.reshape(1, -1)
# print(x_sample)
# print(LogReg.predict(x_sample))

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.9836065573770492

In [26]:
from keras.models import Sequential
from keras.layers import Dense
from keras import metrics
from sklearn.preprocessing import LabelEncoder
from keras.models import model_from_json
import os


model = Sequential()
model.add(Dense(3, input_dim=9, activation='relu'))
model.add(Dense(3, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='sgd', loss='binary_crossentropy', metrics=['accuracy'])

model.summary()


fitModel = model.fit(X_train.values, Y_train.values, batch_size=10, epochs=100, 
                      validation_data=(X_test.values, Y_test.values))



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_7 (Dense)              (None, 3)                 30        
_________________________________________________________________
dense_8 (Dense)              (None, 3)                 12        
_________________________________________________________________
dense_9 (Dense)              (None, 1)                 4         
Total params: 46
Trainable params: 46
Non-trainable params: 0
_________________________________________________________________
Train on 492 samples, validate on 122 samples
Epoch 1/100
492/492 [==============================] - 1s 1ms/step - loss: 0.6407 - acc: 0.6931 - val_loss: 0.6253 - val_acc: 0.6803
Epoch 2/100
492/492 [==============================] - 0s 518us/step - loss: 0.5934 - acc: 0.7520 - val_loss: 0.5804 - val_acc: 0.7295
Epoch 3/100
492/492 [==============================] - 0s 221us/step - loss: 0.5430 - acc: 0.804

492/492 [==============================] - 0s 235us/step - loss: 0.1024 - acc: 0.9858 - val_loss: 0.1025 - val_acc: 0.9836
Epoch 56/100
492/492 [==============================] - 0s 235us/step - loss: 0.1014 - acc: 0.9817 - val_loss: 0.1039 - val_acc: 0.9836
Epoch 57/100
492/492 [==============================] - 0s 208us/step - loss: 0.1003 - acc: 0.9858 - val_loss: 0.1002 - val_acc: 0.9836
Epoch 58/100
492/492 [==============================] - 0s 264us/step - loss: 0.0986 - acc: 0.9858 - val_loss: 0.0997 - val_acc: 0.9836
Epoch 59/100
492/492 [==============================] - 0s 196us/step - loss: 0.0984 - acc: 0.9837 - val_loss: 0.0963 - val_acc: 0.9836
Epoch 60/100
492/492 [==============================] - 0s 218us/step - loss: 0.0960 - acc: 0.9858 - val_loss: 0.0961 - val_acc: 0.9836
Epoch 61/100
492/492 [==============================] - 0s 282us/step - loss: 0.0948 - acc: 0.9858 - val_loss: 0.0939 - val_acc: 0.9836
Epoch 62/100
492/492 [==============================] - 0s 18

In [6]:
# Save model
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")

Saved model to disk


In [23]:
# Function for calculating the probability that the 
# given sample data is malignant.
def calcProb(sample):
    # load json and create model
    json_file = open('model.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)
    # load weights into new model
    loaded_model.load_weights("model.h5")
    print("Loaded model from disk")

    loaded_model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
    loaded_score = loaded_model.evaluate(X_test.values, Y_test.values)
#     print("%s: %.2f%%" % (loaded_model.metrics_names[1], loaded_score[1]*100))

#     for i in range(10,20):
#         print("X_test.value = ")
#         print(X_test.values[i].reshape(1,-1))
#         print("Y_test.value = ")
#         print(Y_test.values[i])
#         print("Predicted = ")
#         print(loaded_model.predict_proba(X_test.values[i].reshape(1,-1), batch_size=1))

    print("Probability of sample being malignant:")
    predictedProb = loaded_model.predict_proba(sample.reshape(1,-1))
    print(predictedProb[0][0])
    return predictedProb[0][0]
        
calcProb(np.array([0, 1, 2, 3, 4, 5, 6, 7, 8]))


Loaded model from disk
122/122 [==============================] - 0s 1ms/step
Probability of sample being malignant:
0.4679505


0.4679505

In [8]:
# filename2 = "breast-cancer_csv.csv"

# # Initialize datafrmae
# dataframe2 = pd.read_csv(filename2, header=0)
# # dataframe = dataframe.drop(0, 1)
# # columns = []
# # dataframe.columns = columns

# print(dataframe2)

       age menopause tumor-size inv-nodes node-caps  deg-malig breast  \
0    40-49   premeno      15-19       0-2       yes          3  right   
1    50-59      ge40      15-19       0-2        no          1  right   
2    50-59      ge40      35-39       0-2        no          2   left   
3    40-49   premeno      35-39       0-2       yes          3  right   
4    40-49   premeno      30-34       3-5       yes          2   left   
5    50-59   premeno      25-29       3-5        no          2  right   
6    50-59      ge40      40-44       0-2        no          3   left   
7    40-49   premeno      10-14       0-2        no          2   left   
8    40-49   premeno        0-4       0-2        no          2  right   
9    40-49      ge40      40-44     15-17       yes          2  right   
10   50-59   premeno      25-29       0-2        no          2   left   
11   60-69      ge40      15-19       0-2        no          2  right   
12   50-59      ge40      30-34       0-2        no